##  Lenet5 Models -JackieN 
This File Produces A number of Lenet5 Models and Predictions based on varying degrees of cleaned Train data.

Based on https://medium.com/@mgazar/lenet-5-in-9-lines-of-code-using-keras-ac99294c8086 and 

https://deepai.org/publication/towards-good-practices-on-building-effective-cnn-baseline-model-for-person-re-identification#:~:text=The%20last%20key%20practice%20is%20to%20train%20CNN,based%20on%20the%20adaptive%20estimates%20of%20lower-order%20moments.

The best score produced from the model using the clean data with all outliers removed is: 3.23581  

Placing at position 72 on the leaderboard

![](https://i.imgur.com/JwXgz3C.jpg)
 

In [1]:
#Set the utils path to point to the utils directory locally
UTILS_PATH = "c:/Users/mspuc/OneDrive/Berkeley/A - W207/Final/blackboxes/utils"

import os, sys
sys.path.append(UTILS_PATH)
from load_models import LoadTrainModels
from predict_models import PredictModels
import imp
import pickle



In [2]:
### this is gold right here.
#import imp
#imp.reload()

#Clean file path
file_path = "C:/Data/CleanTrain_1/"
trainer = LoadTrainModels("C:/data/Jackie_Lenet5", file_path)

trainer.print_paths()

Model dir: C:/data/Jackie_Lenet5/


### For every version of a clean Train file in a given path, create and save a model

In [3]:

#Get the files in the clean directory
files = os.listdir(file_path)

#For every version of a cleaned Train file in CleanTrain directory, create and save a model
for filename in files: 
    print("Opening file: ", filename)
    clean_file = "".join((file_path,filename))
    train_data = pickle.load( open( clean_file, "rb" ) )
    train_data = train_data.drop(['level_0', 'check_sum', 'index'], axis=1,errors='ignore')
    print("Train Shape:", train_data.shape)

    filename = str(filename).replace('.p', '').strip()
    print("Begin model and train:")
    model_name = "".join((filename,"_jn"))
    print("Model name:", model_name)
    model, history = trainer.train_model(model_name, train_data, hoizontal_flip = False,brightness = 1.4, dim = 0.3)
    print("End model and train")    
    print()

Opening file:  clean_all_outliers.p
Train Shape: (7041, 31)
Begin model and train:
Model name: clean_all_outliers_jn
Scaling images
Scaling complete.
Number of images to be brightened: 7041
Number of images to be dimmed: 7041
Completed brighten and dim. Number of observations added to train: 14082
Begining the split of Train with all features
Looking for model JN
Loading model: clean_all_outliers_jn
Saving the history paramters file


NameError: name 'self' is not defined

### For every model file in a given path, predict using the model and save the predictions in CSV file

In [2]:
#Time to make some predictions

id_lookup = pickle.load( open( "../Data/id_lookup.p", "rb" ) )
test = pickle.load( open( "../Data/test.p", "rb" ) )

#Using local paths as this is way faster...
#Where the models are:
file_path = "C:/Data/Jackie_Lenet5"
#Where we want the predictions stored
pred_path = "C:/data/Predictions/"

predictor = PredictModels(file_path,pred_path , id_lookup)

predictor.print_paths()

files = os.listdir(file_path)
#For every model in file_path, predict using the model and save the predictions in CSV file
for filename in files:
    if ".h5" in filename:
        base_name = filename[:-3]
        model_json = ''.join((base_name,".json"))
        print("Working with: ", base_name)
        print("Begin Predict")
        Y= predictor.predict_lenet5(base_name, filename, model_json, test)
        print("End model and train")    
        print()


Model dir: C:/Data/Jackie_Lenet5/
Pickle dir: C:/data/Predictions/
Working with:  clean_all_outliers_Lenet5
Begin Predict
Scaling 1783 images...
Scaling of 1783 observations complete.
Begining the split of Test
got unique ids
test subset shape: (1783, 4)
End with the split of Test
(27124, 4)
before melt: (1783, 30)
after melt: (53490, 3)
after merge: (27124, 2)
C:/data/Predictions/clean_all_outliers_Lenet5Pred.csv
Predictions written 
End model and train

Working with:  clean_duplicates_Lenet5
Begin Predict
Scaling 1783 images...
Scaling of 1783 observations complete.
Begining the split of Test
got unique ids
test subset shape: (1783, 4)
End with the split of Test
(27124, 4)
before melt: (1783, 30)
after melt: (53490, 3)
after merge: (27124, 2)
C:/data/Predictions/clean_duplicates_Lenet5Pred.csv
Predictions written 
End model and train

Working with:  clean_o_dups_Lenet5
Begin Predict
Scaling 1783 images...
Scaling of 1783 observations complete.
Begining the split of Test
got unique id